# Check 2025 dataset size

In [1]:
import os
from dataset import train_test_split_indicies
from transformer_trainer import BrainToTextDecoder_Trainer
from omegaconf import OmegaConf

args = OmegaConf.load('transformer_args.yaml')

/opt/SoftOpen/anaconda3/envs/b2txt25/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/SoftOpen/anaconda3/envs/b2txt25/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# In your training script, after creating the trainer:
# trainer = BrainToTextDecoder_Trainer(args)

# Access the train_trials dictionary (it's created but not stored as instance variable)
# You'd need to modify the trainer to expose this, or recreate it:

train_file_paths = [os.path.join(args["dataset"]["dataset_dir"], s, 'data_train.hdf5') 
                   for s in args['dataset']['sessions']]

train_trials, _ = train_test_split_indicies(
    file_paths=train_file_paths, 
    test_percentage=0,
    seed=args['dataset']['seed'],
    bad_trials_dict=None,
)

# Count total trials across all days
total_trials = 0
for day_idx, day_info in train_trials.items():
    num_trials_this_day = len(day_info['trials'])
    total_trials += num_trials_this_day
    print(f"Day {day_idx}: {num_trials_this_day} trials")

print(f"Total training samples across all days: {total_trials}")

Day 0: 288 trials
Day 1: 348 trials
Day 2: 197 trials
Day 3: 278 trials
Day 4: 88 trials
Day 5: 150 trials
Day 6: 297 trials
Day 7: 322 trials
Day 8: 245 trials
Day 9: 153 trials
Day 10: 218 trials
Day 11: 174 trials
Day 12: 284 trials
Day 13: 155 trials
Day 14: 239 trials
Day 15: 98 trials
Day 16: 134 trials
Day 17: 149 trials
Day 18: 80 trials
Day 19: 100 trials
Day 20: 60 trials
Day 21: 198 trials
Day 22: 228 trials
Day 23: 198 trials
Day 24: 131 trials
Day 25: 135 trials
Day 26: 198 trials
Day 27: 193 trials
Day 28: 219 trials
Day 29: 163 trials
Day 30: 239 trials
Day 31: 246 trials
Day 32: 364 trials
Day 33: 150 trials
Day 34: 110 trials
Day 35: 90 trials
Day 36: 169 trials
Day 37: 160 trials
Day 38: 161 trials
Day 39: 106 trials
Day 40: 163 trials
Day 41: 59 trials
Day 42: 101 trials
Day 43: 165 trials
Day 44: 69 trials
Total training samples across all days: 8072


In [4]:
val_file_paths = [os.path.join(args["dataset"]["dataset_dir"],s,'data_val.hdf5') for s in args['dataset']['sessions']]

_, val_trials = train_test_split_indicies(
            file_paths = val_file_paths, 
            test_percentage = 1,
            seed = args['dataset']['seed'],
            bad_trials_dict = None,
            )

# Count total trials in validation set
total_val_trials = 0
for day_idx, day_info in val_trials.items():
    num_trials_this_day = len(day_info['trials'])
    total_val_trials += num_trials_this_day
    print(f"Day {day_idx}: {num_trials_this_day} trials")

print(f"Total validation samples across all days: {total_val_trials}")

Day 0: 0 trials
Day 1: 35 trials
Day 2: 49 trials
Day 3: 48 trials
Day 4: 25 trials
Day 5: 25 trials
Day 6: 49 trials
Day 7: 34 trials
Day 8: 35 trials
Day 9: 48 trials
Day 10: 44 trials
Day 11: 36 trials
Day 12: 17 trials
Day 13: 44 trials
Day 14: 44 trials
Day 15: 9 trials
Day 16: 33 trials
Day 17: 50 trials
Day 18: 15 trials
Day 19: 25 trials
Day 20: 20 trials
Day 21: 44 trials
Day 22: 34 trials
Day 23: 50 trials
Day 24: 25 trials
Day 25: 30 trials
Day 26: 50 trials
Day 27: 23 trials
Day 28: 0 trials
Day 29: 24 trials
Day 30: 48 trials
Day 31: 48 trials
Day 32: 0 trials
Day 33: 0 trials
Day 34: 25 trials
Day 35: 25 trials
Day 36: 48 trials
Day 37: 46 trials
Day 38: 48 trials
Day 39: 23 trials
Day 40: 47 trials
Day 41: 24 trials
Day 42: 24 trials
Day 43: 30 trials
Day 44: 25 trials
Total validation samples across all days: 1426


# Check dataset format

In [5]:
import os
import torch
from torch.utils.data import Dataset 
import h5py
import numpy as np
from torch.nn.utils.rnn import pad_sequence
import math 

In [ ]:
batch = {
            'input_features' : [],
            'seq_class_ids' : [],
            'n_time_steps' : [],
            'phone_seq_lens' : [],
            'day_indicies' : [],
            'transcriptions' : [],
            'block_nums' : [],
            'trial_nums' : [],
        }

with h5py.File('/net/inlsrv2.epfl.ch/scratch/jhliu/nejm-brain-to-text-main/data/hdf5_data_final/t15.2023.08.11/data_train.hdf5', 'r') as f:
    # For each trial in the selected trials in that day
    d = 0
    g = f[f'trial_{d:04d}']

    # Remove features is neccessary 
    input_features = torch.from_numpy(g['input_features'][:]) # neural data
    print(g['input_features'][:].shape)
    print(g['seq_class_ids'][:].shape, g['seq_class_ids'][:])
    print(g['transcription'][:].shape, g['transcription'][:])
    print(g.attrs['n_time_steps'])
    print(g.attrs['seq_len'])
    print(g.attrs['block_num'])
    print(g.attrs['trial_num'])
    print(g)

    batch['input_features'].append(input_features)

    batch['seq_class_ids'].append(torch.from_numpy(g['seq_class_ids'][:]))  # phoneme labels
    batch['transcriptions'].append(torch.from_numpy(g['transcription'][:])) # character level transcriptions
    batch['n_time_steps'].append(g.attrs['n_time_steps']) # number of time steps in the trial - required since we are padding
    batch['phone_seq_lens'].append(g.attrs['seq_len']) # number of phonemes in the label - required since we are padding
    batch['day_indicies'].append(int(d)) # day index of each trial - required for the day specific layers 
    batch['block_nums'].append(g.attrs['block_num'])
    batch['trial_nums'].append(g.attrs['trial_num'])

(321, 512)
(500,) [ 7 28 17 24 40 17 31 40 20 21 25 29 12 40  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0

In [24]:
# ARPAbet phoneme set (common in speech recognition)
phoneme_vocab = {
    0: '<blank>',  # CTC blank token
    1: 'AA', 2: 'AE', 3: 'AH', 4: 'AO', 5: 'AW', 6: 'AY',
    7: 'B', 8: 'CH', 9: 'D', 10: 'DH', 11: 'EH', 12: 'ER',
    13: 'EY', 14: 'F', 15: 'G', 16: 'HH', 17: 'IH', 18: 'IY',
    19: 'JH', 20: 'K', 21: 'L', 22: 'M', 23: 'N', 24: 'NG',
    25: 'OW', 26: 'OY', 27: 'P', 28: 'R', 29: 'S', 30: 'SH',
    31: 'T', 32: 'TH', 33: 'UH', 34: 'UW', 35: 'V', 36: 'W',
    37: 'Y', 38: 'Z', 39: 'ZH', 40: '<sil>'  # silence
}

# Convert phoneme IDs to phoneme characters
with h5py.File('/net/inlsrv2.epfl.ch/scratch/jhliu/nejm-brain-to-text-main/data/hdf5_data_final/t15.2023.08.11/data_train.hdf5', 'r') as f:
    trial_key = 'trial_0000'
    
    # Get phoneme IDs
    phoneme_ids = f[trial_key]['seq_class_ids'][:]
    print(f"Phoneme IDs: {phoneme_ids}")
    
    # Convert to phoneme characters
    phoneme_chars = [phoneme_vocab[id] for id in phoneme_ids if id in phoneme_vocab]
    print(f"Phoneme characters: {phoneme_chars}")
    
    # Join into a string (optional)
    phoneme_sequence = ' '.join(phoneme_chars)
    print(f"Phoneme sequence: {phoneme_sequence}")

Phoneme IDs: [ 7 28 17 24 40 17 31 40 20 21 25 29 12 40  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  

In [23]:
with h5py.File('/net/inlsrv2.epfl.ch/scratch/jhliu/nejm-brain-to-text-main/data/hdf5_data_final/t15.2023.08.11/data_train.hdf5', 'r') as f:
    print("=== FILE ATTRIBUTES ===")
    for key, value in f.attrs.items():
        print(f"  {key}: {value}")
    
    print("\n=== ALL DATASETS AND GROUPS ===")
    def detailed_print(name, obj):
        indent = "  " * name.count('/')
        if isinstance(obj, h5py.Dataset):
            print(f"{indent}📄 {name} - Shape: {obj.shape}, Type: {obj.dtype}")
            # Show attributes if any
            if obj.attrs:
                for attr_key, attr_val in obj.attrs.items():
                    print(f"{indent}    📎 {attr_key}: {attr_val}")
        elif isinstance(obj, h5py.Group):
            print(f"{indent}📁 {name}/")
            # Show group attributes if any
            if obj.attrs:
                for attr_key, attr_val in obj.attrs.items():
                    print(f"{indent}    📎 {attr_key}: {attr_val}")
    
    f.visititems(detailed_print)

=== FILE ATTRIBUTES ===

=== ALL DATASETS AND GROUPS ===
📁 trial_0000/
    📎 block_num: 2
    📎 n_time_steps: 321
    📎 sentence_label: Bring it closer.
    📎 seq_len: 14
    📎 session: t15.2023.08.11
    📎 trial_num: 0
  📄 trial_0000/input_features - Shape: (321, 512), Type: float32
  📄 trial_0000/seq_class_ids - Shape: (500,), Type: int32
  📄 trial_0000/transcription - Shape: (500,), Type: int32
📁 trial_0001/
    📎 block_num: 2
    📎 n_time_steps: 481
    📎 sentence_label: My family is closer.
    📎 seq_len: 19
    📎 session: t15.2023.08.11
    📎 trial_num: 1
  📄 trial_0001/input_features - Shape: (481, 512), Type: float32
  📄 trial_0001/seq_class_ids - Shape: (500,), Type: int32
  📄 trial_0001/transcription - Shape: (500,), Type: int32
📁 trial_0002/
    📎 block_num: 2
    📎 n_time_steps: 480
    📎 sentence_label: What do they like?
    📎 seq_len: 14
    📎 session: t15.2023.08.11
    📎 trial_num: 2
  📄 trial_0002/input_features - Shape: (480, 512), Type: float32
  📄 trial_0002/seq_cl

In [18]:
with h5py.File('/net/inlsrv2.epfl.ch/scratch/jhliu/nejm-brain-to-text-main/data/hdf5_data_final/t15.2023.08.11/data_train.hdf5', 'r') as f:
    trial_key = 'trial_0000'  # Example trial
    
    # Phoneme sequence (class IDs)
    phoneme_ids = f[trial_key]['seq_class_ids'][:]
    print(f"Phoneme IDs: {phoneme_ids}")
    
    # Actual transcription (characters)
    transcription_codes = f[trial_key]['transcription'][:]
    text = ''.join([chr(c) for c in transcription_codes if c != 0])
    print(f"Spoken text: '{text}'")

Phoneme IDs: [ 7 28 17 24 40 17 31 40 20 21 25 29 12 40  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  

# B2txt24 data

In [83]:
import os
import pickle
import time
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

In [84]:
with open('/net/inlsrv2.epfl.ch/scratch/jhliu/Public_Datasets/Human/doi_10_5061_dryad_x69p8czpq__v20230901/ptDecoder_ctc', "rb") as handle:
    loadedData = pickle.load(handle)

In [85]:
loadedData['train'][0].keys()

dict_keys(['sentenceDat', 'transcriptions', 'phonemes', 'timeSeriesLens', 'phoneLens', 'phonePerTime'])

In [78]:
print(loadedData['train'][0]['sentenceDat'][0].shape)
print(loadedData['train'][0]['transcriptions'][0])
print(loadedData['train'][0]['phonemes'][0].shape)
print(loadedData['train'][0]['timeSeriesLens'][0])
print(loadedData['train'][0]['phoneLens'][0])
print(loadedData['train'][0]['phonePerTime'][0])

(478, 256)
Nuclear rockets can destroy airfields with ease.
(500,)
478
40
0.08368201


In [52]:
# First, let's understand the B2TXT24 structure
print("=== B2TXT24 DATA STRUCTURE ===")
print(f"loadedData keys: {list(loadedData.keys())}")
print(f"Train data type: {type(loadedData['train'])}")

# Check if train is a list of sessions or a dictionary
if isinstance(loadedData['train'], list):
    print(f"Number of training sessions: {len(loadedData['train'])}")
    print(f"First session keys: {list(loadedData['train'][0].keys())}")
    
    # Examine first session structure
    session = loadedData['train'][0]
    for key, value in session.items():
        if hasattr(value, '__len__'):
            print(f"  {key}: length={len(value)}, type={type(value)}")
            if len(value) > 0:
                print(f"    First item shape: {getattr(value[0], 'shape', 'N/A')}")

=== B2TXT24 DATA STRUCTURE ===
loadedData keys: ['train', 'test', 'competition']
Train data type: <class 'list'>
Number of training sessions: 24
First session keys: ['sentenceDat', 'transcriptions', 'phonemes', 'timeSeriesLens', 'phoneLens', 'phonePerTime']
  sentenceDat: length=280, type=<class 'list'>
    First item shape: (478, 256)
  transcriptions: length=280, type=<class 'list'>
    First item shape: N/A
  phonemes: length=280, type=<class 'list'>
    First item shape: (500,)
  timeSeriesLens: length=280, type=<class 'numpy.ndarray'>
    First item shape: ()
  phoneLens: length=280, type=<class 'numpy.ndarray'>
    First item shape: ()
  phonePerTime: length=280, type=<class 'numpy.ndarray'>
    First item shape: ()


In [65]:
trainsessionNames = ['t12.2022.04.28',  't12.2022.05.26',  't12.2022.06.21',  't12.2022.07.21',  't12.2022.08.13',
't12.2022.05.05',  't12.2022.06.02',  't12.2022.06.23',  't12.2022.07.27',  't12.2022.08.18',
't12.2022.05.17',  't12.2022.06.07',  't12.2022.06.28',  't12.2022.07.29',  't12.2022.08.23',
't12.2022.05.19',  't12.2022.06.14',  't12.2022.07.05',  't12.2022.08.02',  't12.2022.08.25',
't12.2022.05.24',  't12.2022.06.16',  't12.2022.07.14',  't12.2022.08.11']
trainsessionNames.sort()
print(trainsessionNames)

['t12.2022.04.28', 't12.2022.05.05', 't12.2022.05.17', 't12.2022.05.19', 't12.2022.05.24', 't12.2022.05.26', 't12.2022.06.02', 't12.2022.06.07', 't12.2022.06.14', 't12.2022.06.16', 't12.2022.06.21', 't12.2022.06.23', 't12.2022.06.28', 't12.2022.07.05', 't12.2022.07.14', 't12.2022.07.21', 't12.2022.07.27', 't12.2022.07.29', 't12.2022.08.02', 't12.2022.08.11', 't12.2022.08.13', 't12.2022.08.18', 't12.2022.08.23', 't12.2022.08.25']


In [66]:
valsessionNames = ['t12.2022.04.28',  't12.2022.05.26',  't12.2022.06.21',  't12.2022.07.21',  't12.2022.08.13',
't12.2022.05.05',  't12.2022.06.02',  't12.2022.06.23',  't12.2022.07.27',  't12.2022.08.18',
't12.2022.05.17',  't12.2022.06.07',  't12.2022.06.28',  't12.2022.07.29',  't12.2022.08.23',
't12.2022.05.19',  't12.2022.06.14',  't12.2022.07.05',  't12.2022.08.02',  't12.2022.08.25',
't12.2022.05.24',  't12.2022.06.16',  't12.2022.07.14',  't12.2022.08.11']
valsessionNames.sort()
print(valsessionNames)

['t12.2022.04.28', 't12.2022.05.05', 't12.2022.05.17', 't12.2022.05.19', 't12.2022.05.24', 't12.2022.05.26', 't12.2022.06.02', 't12.2022.06.07', 't12.2022.06.14', 't12.2022.06.16', 't12.2022.06.21', 't12.2022.06.23', 't12.2022.06.28', 't12.2022.07.05', 't12.2022.07.14', 't12.2022.07.21', 't12.2022.07.27', 't12.2022.07.29', 't12.2022.08.02', 't12.2022.08.11', 't12.2022.08.13', 't12.2022.08.18', 't12.2022.08.23', 't12.2022.08.25']


In [67]:
testsessionNames = ['t12.2022.05.26',  't12.2022.06.21',  't12.2022.07.21',  't12.2022.08.13',
't12.2022.06.02',  't12.2022.07.27',
't12.2022.06.07',  't12.2022.06.28',
't12.2022.06.14',  't12.2022.07.05',  't12.2022.08.02',
't12.2022.05.24',  't12.2022.06.16',  't12.2022.07.14',  't12.2022.08.11']
testsessionNames.sort()
print(testsessionNames)

['t12.2022.05.24', 't12.2022.05.26', 't12.2022.06.02', 't12.2022.06.07', 't12.2022.06.14', 't12.2022.06.16', 't12.2022.06.21', 't12.2022.06.28', 't12.2022.07.05', 't12.2022.07.14', 't12.2022.07.21', 't12.2022.07.27', 't12.2022.08.02', 't12.2022.08.11', 't12.2022.08.13']


In [79]:
import h5py
import numpy as np
import os
from pathlib import Path

def convert_b2txt24_to_hdf5(pickle_data, output_dir, transcription_max_len=500):
    """
    Convert B2TXT24 pickle format to B2TXT25 HDF5 format
    """
    
    # Create output directory
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    def pad_transcription(transcription, max_len=transcription_max_len):
        """Convert string to ASCII codes and pad to fixed length"""
        transcription_codes = np.array([ord(c) for c in transcription])
        
        # Pad or truncate to max_len
        if len(transcription_codes) < max_len:
            # Pad with zeros
            padded = np.zeros(max_len, dtype=np.int32)
            padded[:len(transcription_codes)] = transcription_codes
            return padded
        else:
            # Truncate if too long
            return transcription_codes[:max_len].astype(np.int32)
    
    # Process training data
    if 'train' in pickle_data:
        train_sessions = pickle_data['train']
        
        for session_idx, session_data in enumerate(train_sessions):
            print(f"Converting training session {session_idx}...")
            
            # Create HDF5 file for this session
            # session_dir = os.path.join(output_dir, f'session_{session_idx:02d}')
            session_dir = os.path.join(output_dir, trainsessionNames[session_idx])
            os.makedirs(session_dir, exist_ok=True)
            
            train_file = os.path.join(session_dir, 'data_train.hdf5')
            
            with h5py.File(train_file, 'w') as f:
                # Convert each trial
                sentence_data = session_data['sentenceDat']
                transcriptions = session_data['transcriptions']
                phonemes = session_data['phonemes']
                time_lens = session_data['timeSeriesLens']
                phone_lens = session_data['phoneLens']
                
                num_trials = len(sentence_data)
                
                for trial_idx in range(num_trials):
                    trial_group = f.create_group(f'trial_{trial_idx:04d}')
                    
                    # Neural data (equivalent to input_features in B2TXT25)
                    neural_data = sentence_data[trial_idx]
                    trial_group.create_dataset('input_features', data=neural_data)
                    
                    # Phoneme labels (equivalent to seq_class_ids)
                    phoneme_ids = phonemes[trial_idx]
                    trial_group.create_dataset('seq_class_ids', data=phoneme_ids)
                    
                    # Transcription (character-level)
                    transcription = transcriptions[trial_idx]
                    transcription_codes = pad_transcription(transcription, transcription_max_len)
                    trial_group.create_dataset('transcription', data=transcription_codes)
                    
                    # Attributes (metadata)
                    trial_group.attrs['n_time_steps'] = time_lens[trial_idx]
                    trial_group.attrs['seq_len'] = phone_lens[trial_idx]
                    trial_group.attrs['block_num'] = 0  # Default if not available
                    trial_group.attrs['trial_num'] = trial_idx
                
                print(f"Saved {num_trials} trials to {train_file}")
    
    # Process validation data if available
    if 'test' in pickle_data:
        val_sessions = pickle_data['test']
        
        for session_idx, session_data in enumerate(val_sessions):
            print(f"Converting validation session {session_idx}...")
            
            # session_dir = os.path.join(output_dir, f'session_{session_idx:02d}')
            session_dir = os.path.join(output_dir, valsessionNames[session_idx])
            os.makedirs(session_dir, exist_ok=True)
            
            val_file = os.path.join(session_dir, 'data_val.hdf5')
            
            with h5py.File(val_file, 'w') as f:
                # Convert each trial
                sentence_data = session_data['sentenceDat']
                transcriptions = session_data['transcriptions']
                phonemes = session_data['phonemes']
                time_lens = session_data['timeSeriesLens']
                phone_lens = session_data['phoneLens']
                
                num_trials = len(sentence_data)
                
                for trial_idx in range(num_trials):
                    trial_group = f.create_group(f'trial_{trial_idx:04d}')
                    
                    # Neural data (equivalent to input_features in B2TXT25)
                    neural_data = sentence_data[trial_idx]
                    trial_group.create_dataset('input_features', data=neural_data)
                    
                    # Phoneme labels (equivalent to seq_class_ids)
                    phoneme_ids = phonemes[trial_idx]
                    trial_group.create_dataset('seq_class_ids', data=phoneme_ids)
                    
                    # Transcription (character-level)
                    transcription = transcriptions[trial_idx]
                    transcription_codes = pad_transcription(transcription, transcription_max_len)
                    trial_group.create_dataset('transcription', data=transcription_codes)
                    
                    # Attributes (metadata)
                    trial_group.attrs['n_time_steps'] = time_lens[trial_idx]
                    trial_group.attrs['seq_len'] = phone_lens[trial_idx]
                    trial_group.attrs['block_num'] = 0  # Default if not available
                    trial_group.attrs['trial_num'] = trial_idx
                
                print(f"Saved {num_trials} trials to {val_file}")
    
    # Process validation data if available
    if 'competition' in pickle_data:
        test_sessions = pickle_data['competition']
        
        for session_idx, session_data in enumerate(test_sessions):
            print(f"Converting testing session {session_idx}...")
            
            # session_dir = os.path.join(output_dir, f'session_{session_idx:02d}')
            session_dir = os.path.join(output_dir, testsessionNames[session_idx])
            os.makedirs(session_dir, exist_ok=True)
            
            test_file = os.path.join(session_dir, 'data_test.hdf5')
            
            with h5py.File(test_file, 'w') as f:
                # Convert each trial
                sentence_data = session_data['sentenceDat']
                transcriptions = session_data['transcriptions']
                phonemes = session_data['phonemes']
                time_lens = session_data['timeSeriesLens']
                phone_lens = session_data['phoneLens']
                
                num_trials = len(sentence_data)
                
                for trial_idx in range(num_trials):
                    trial_group = f.create_group(f'trial_{trial_idx:04d}')
                    
                    # Neural data (equivalent to input_features in B2TXT25)
                    neural_data = sentence_data[trial_idx]
                    trial_group.create_dataset('input_features', data=neural_data)
                    
                    # Phoneme labels (equivalent to seq_class_ids)
                    phoneme_ids = phonemes[trial_idx]
                    trial_group.create_dataset('seq_class_ids', data=phoneme_ids)
                    
                    # Transcription (character-level)
                    transcription = transcriptions[trial_idx]
                    transcription_codes = pad_transcription(transcription, transcription_max_len)
                    trial_group.create_dataset('transcription', data=transcription_codes)
                    
                    # Attributes (metadata)
                    trial_group.attrs['n_time_steps'] = time_lens[trial_idx]
                    trial_group.attrs['seq_len'] = phone_lens[trial_idx]
                    trial_group.attrs['block_num'] = 0  # Default if not available
                    trial_group.attrs['trial_num'] = trial_idx
                
                print(f"Saved {num_trials} trials to {test_file}")

# Use the conversion function
output_directory = '/net/inlsrv2.epfl.ch/scratch/jhliu/Public_Datasets/Human/doi_10_5061_dryad_x69p8czpq__v20230901/converted/b2txt24_hdf5'
convert_b2txt24_to_hdf5(loadedData, output_directory, transcription_max_len=500)

Converting training session 0...
Saved 280 trials to /net/inlsrv2.epfl.ch/scratch/jhliu/Public_Datasets/Human/doi_10_5061_dryad_x69p8czpq__v20230901/converted/b2txt24_hdf5/t12.2022.04.28/data_train.hdf5
Converting training session 1...
Saved 360 trials to /net/inlsrv2.epfl.ch/scratch/jhliu/Public_Datasets/Human/doi_10_5061_dryad_x69p8czpq__v20230901/converted/b2txt24_hdf5/t12.2022.05.05/data_train.hdf5
Converting training session 2...
Saved 420 trials to /net/inlsrv2.epfl.ch/scratch/jhliu/Public_Datasets/Human/doi_10_5061_dryad_x69p8czpq__v20230901/converted/b2txt24_hdf5/t12.2022.05.17/data_train.hdf5
Converting training session 3...
Saved 180 trials to /net/inlsrv2.epfl.ch/scratch/jhliu/Public_Datasets/Human/doi_10_5061_dryad_x69p8czpq__v20230901/converted/b2txt24_hdf5/t12.2022.05.19/data_train.hdf5
Converting training session 4...
Saved 360 trials to /net/inlsrv2.epfl.ch/scratch/jhliu/Public_Datasets/Human/doi_10_5061_dryad_x69p8czpq__v20230901/converted/b2txt24_hdf5/t12.2022.05.24/d

In [80]:
batch = {
            'input_features' : [],
            'seq_class_ids' : [],
            'n_time_steps' : [],
            'phone_seq_lens' : [],
            'day_indicies' : [],
            'transcriptions' : [],
            'block_nums' : [],
            'trial_nums' : [],
        }

with h5py.File('/net/inlsrv2.epfl.ch/scratch/jhliu/Public_Datasets/Human/doi_10_5061_dryad_x69p8czpq__v20230901/converted/b2txt24_hdf5/t12.2022.05.24/data_train.hdf5', 'r') as f:
    # For each trial in the selected trials in that day
    d = 0
    g = f[f'trial_{d:04d}']

    # Remove features is neccessary 
    input_features = torch.from_numpy(g['input_features'][:]) # neural data
    print(g['input_features'][:].shape)
    print(g['seq_class_ids'][:].shape, g['seq_class_ids'][:])
    print(g['transcription'][:].shape, g['transcription'][:])
    print(g.attrs['n_time_steps'])
    print(g.attrs['seq_len'])
    print(g.attrs['block_num'])
    print(g.attrs['trial_num'])
    print(g)

    batch['input_features'].append(input_features)

    batch['seq_class_ids'].append(torch.from_numpy(g['seq_class_ids'][:]))  # phoneme labels
    batch['transcriptions'].append(torch.from_numpy(g['transcription'][:])) # character level transcriptions
    batch['n_time_steps'].append(g.attrs['n_time_steps']) # number of time steps in the trial - required since we are padding
    batch['phone_seq_lens'].append(g.attrs['seq_len']) # number of phonemes in the label - required since we are padding
    batch['day_indicies'].append(int(d)) # day index of each trial - required for the day specific layers 
    batch['block_nums'].append(g.attrs['block_num'])
    batch['trial_nums'].append(g.attrs['trial_num'])

(252, 256)
(500,) [10 13 40 30 33  9 40 28 17 21 18 40  7 18 40  5 31 21  4  9 40  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0

In [81]:
# ARPAbet phoneme set (common in speech recognition)
phoneme_vocab = {
    0: '<blank>',  # CTC blank token
    1: 'AA', 2: 'AE', 3: 'AH', 4: 'AO', 5: 'AW', 6: 'AY',
    7: 'B', 8: 'CH', 9: 'D', 10: 'DH', 11: 'EH', 12: 'ER',
    13: 'EY', 14: 'F', 15: 'G', 16: 'HH', 17: 'IH', 18: 'IY',
    19: 'JH', 20: 'K', 21: 'L', 22: 'M', 23: 'N', 24: 'NG',
    25: 'OW', 26: 'OY', 27: 'P', 28: 'R', 29: 'S', 30: 'SH',
    31: 'T', 32: 'TH', 33: 'UH', 34: 'UW', 35: 'V', 36: 'W',
    37: 'Y', 38: 'Z', 39: 'ZH', 40: '<sil>'  # silence
}

# Convert phoneme IDs to phoneme characters
with h5py.File('/net/inlsrv2.epfl.ch/scratch/jhliu/Public_Datasets/Human/doi_10_5061_dryad_x69p8czpq__v20230901/converted/b2txt24_hdf5/t12.2022.05.24/data_train.hdf5', 'r') as f:
    trial_key = 'trial_0000'
    
    # Get phoneme IDs
    phoneme_ids = f[trial_key]['seq_class_ids'][:]
    print(f"Phoneme IDs: {phoneme_ids}")
    
    # Convert to phoneme characters
    phoneme_chars = [phoneme_vocab[id] for id in phoneme_ids if id in phoneme_vocab]
    print(f"Phoneme characters: {phoneme_chars}")
    
    # Join into a string (optional)
    phoneme_sequence = ' '.join(phoneme_chars)
    print(f"Phoneme sequence: {phoneme_sequence}")

Phoneme IDs: [10 13 40 30 33  9 40 28 17 21 18 40  7 18 40  5 31 21  4  9 40  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  

In [82]:
with h5py.File('/net/inlsrv2.epfl.ch/scratch/jhliu/Public_Datasets/Human/doi_10_5061_dryad_x69p8czpq__v20230901/converted/b2txt24_hdf5/t12.2022.05.24/data_train.hdf5', 'r') as f:
    trial_key = 'trial_0000'  # Example trial
    
    # Phoneme sequence (class IDs)
    phoneme_ids = f[trial_key]['seq_class_ids'][:]
    print(f"Phoneme IDs: {phoneme_ids}")
    
    # Actual transcription (characters)
    transcription_codes = f[trial_key]['transcription'][:]
    text = ''.join([chr(c) for c in transcription_codes if c != 0])
    print(f"Spoken text: '{text}'")

Phoneme IDs: [10 13 40 30 33  9 40 28 17 21 18 40  7 18 40  5 31 21  4  9 40  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  

# Check ROPE and register tokens integration

In [1]:
from transformer_model import TransformerDecoder

In [2]:
model = TransformerDecoder()

In [7]:
model.freqs_cis[:500]

tensor([[ 1.0000+0.0000e+00j,  1.0000+0.0000e+00j,  1.0000+0.0000e+00j,
          ...,  1.0000+0.0000e+00j,  1.0000+0.0000e+00j,
          1.0000+0.0000e+00j],
        [ 0.5403+8.4147e-01j,  0.7318+6.8156e-01j,  0.8460+5.3317e-01j,
          ...,  1.0000+2.3714e-04j,  1.0000+1.7783e-04j,
          1.0000+1.3335e-04j],
        [-0.4161+9.0930e-01j,  0.0709+9.9748e-01j,  0.4315+9.0213e-01j,
          ...,  1.0000+4.7427e-04j,  1.0000+3.5566e-04j,
          1.0000+2.6670e-04j],
        ...,
        [ 0.8090+5.8782e-01j, -0.4066+9.1362e-01j, -0.9930+1.1783e-01j,
          ...,  0.9931+1.1758e-01j,  0.9961+8.8265e-02j,
          0.9978+6.6228e-02j],
        [-0.0575+9.9834e-01j, -0.9202+3.9144e-01j, -0.9029-4.2978e-01j,
          ...,  0.9930+1.1782e-01j,  0.9961+8.8443e-02j,
          0.9978+6.6361e-02j],
        [-0.8712+4.9100e-01j, -0.9402-3.4072e-01j, -0.5347-8.4501e-01j,
          ...,  0.9930+1.1806e-01j,  0.9961+8.8620e-02j,
          0.9978+6.6494e-02j]])

In [8]:
model.freqs_cis[0]

tensor([1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
        1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
        1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])